# DataProcessing Notebook - AAAG

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
pd.set_option('display.max_columns', None)
df_train=pd.read_csv('Version_5_df.csv',index_col='Id')

In [2]:
df_train.sample(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69,30,RM,47.0,4608,Pave,No Alley,Reg,Lvl,Corner,Gtl,OldTown,Artery,Norm,1Fam,1Story,4,6,1945,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,4,CBlock,3,3,1,Unf,747,GasA,3,Y,SBrkr,747,0,747,0,0,1,0,2,3,4,Typ,0,0,Attchd,1945.0,Unf,1,220,3,3,Y,0,0,0,0,0,0,No Fence,No Shed,6,2010,WD,Normal,11.289794
980,20,RL,80.0,8816,Pave,No Alley,Reg,Lvl,Corner,Gtl,Sawyer,Feedr,Norm,1Fam,1Story,5,6,1963,1963,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,3,PConc,3,3,1,Rec,1121,GasA,3,Y,SBrkr,1121,0,1121,1,0,1,0,3,3,5,Typ,0,0,Detchd,1963.0,Unf,2,480,3,3,Y,0,1,0,0,0,0,MnPrv,No Shed,6,2009,WD,Normal,11.842236
850,80,RL,80.0,9600,Pave,No Alley,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,SLvl,6,7,1976,1994,Hip,CompShg,Plywood,Plywood,BrkFace,360.0,4,4,CBlock,3,3,1,Unf,528,GasA,5,Y,SBrkr,1094,761,1855,0,0,2,1,3,3,7,Typ,1,3,Attchd,1976.0,RFn,2,512,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,8,2007,WD,Normal,12.138869
1102,20,RL,61.0,9758,Pave,No Alley,IR1,Lvl,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1971,1971,Gable,CompShg,HdBoard,MetalSd,None,0.0,3,3,CBlock,3,3,1,BLQ,950,GasA,3,Y,SBrkr,950,0,950,0,0,1,0,3,3,5,Typ,0,0,Detchd,1981.0,Unf,1,280,3,3,Y,0,0,0,0,0,0,No Fence,No Shed,7,2007,WD,Normal,11.691080
388,80,RL,72.0,7200,Pave,No Alley,Reg,Lvl,Inside,Gtl,Edwards,Norm,Norm,1Fam,SLvl,6,6,1976,1976,Hip,CompShg,MetalSd,MetalSd,BrkFace,255.0,3,3,CBlock,3,3,3,ALQ,1041,GasA,5,Y,SBrkr,1125,0,1125,1,0,1,0,3,3,6,Typ,1,2,Detchd,1977.0,Unf,1,352,3,3,Y,1,0,0,0,0,0,GdWo,No Shed,10,2009,WD,Abnorml,11.736077
7,20,RL,75.0,10084,Pave,No Alley,Reg,Lvl,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,4,3,PConc,5,3,3,GLQ,1686,GasA,5,Y,SBrkr,1694,0,1694,1,0,2,0,3,4,7,Typ,1,4,Attchd,2004.0,RFn,2,636,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,8,2007,WD,Normal,12.634606
1288,20,RL,80.0,36500,Pave,No Alley,IR1,Low,Inside,Mod,ClearCr,Norm,Norm,1Fam,1Story,5,5,1964,1964,Gable,CompShg,Wd Sdng,Wd Sdng,BrkCmn,621.0,3,4,CBlock,3,3,3,Rec,1624,GasA,2,Y,SBrkr,1582,0,1582,0,1,2,0,4,3,7,Typ,0,0,Attchd,1964.0,Unf,2,390,3,3,N,1,1,0,0,0,0,No Fence,No Shed,6,2006,WD,Normal,12.154785
1441,70,RL,79.0,11526,Pave,No Alley,IR1,Bnk,Inside,Mod,Crawfor,Norm,Norm,1Fam,2.5Fin,6,7,1922,1994,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,BrkTil,5,3,1,Unf,588,GasA,2,Y,SBrkr,1423,748,2555,0,0,2,0,3,3,11,Min1,1,4,Detchd,1993.0,Fin,2,672,3,3,Y,1,0,0,0,0,0,No Fence,No Shed,9,2008,WD,Normal,12.160034
578,80,RL,96.0,11777,Pave,No Alley,IR1,Lvl,Inside,Gtl,Sawyer,Norm,Norm,1Fam,SLvl,5,6,1966,1966,Gable,CompShg,VinylSd,VinylSd,BrkFace,97.0,3,3,CBlock,3,3,3,LwQ,1164,GasA,5,Y,SBrkr,1320,0,1320,1,0,1,0,3,3,6,Typ,2,2,Attchd,1966.0,RFn,2,564,3,3,Y,1,1,1,0,0,0,No Fence,No Shed,5,2006,WD,Abnorml,12.010672


In [3]:
###########################################################################
#### Just for reference #####
###########################################################################
# Original category type casting: 
nominal_var=['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','BsmtFinType2','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass']
ordinal_var=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath',\
             'HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars',\
             'MoSold','YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond','PoolQC']

cont_var=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',\
          '1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF',\
          'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']

var_dict={'nominal_var':nominal_var,'ordinal_var':ordinal_var,'cont_var':cont_var}
###########################################################################
#### Just for reference #####
###########################################################################


In [4]:
def feature_standardize(data,scaleType='standardize'):
    '''
    - Accepts a dataframe column
    '''
    if scaleType not in ['standardize', 'normalize']: 
        raise ValueError('%s is not a valid choice' %(scaleType))
    mean_value=np.mean(data)
    standard_dev=np.std(data)
    min_value=np.min(data)
    max_value=np.min(data)
    if scaleType == 'standardize':
        return((data-mean_value)/standard_dev) 
    elif scaleType == 'normalize':
        return((data-min_value)/(max_value-min_value))

In [5]:
def dummify_column(dataframe,column_name):
    '''
    ### NOTE!!! ### vector operation not working yet!!!! ###
    - dataframe takes the entire dataframe you are working on
    - column_name takes a list of strings, where the strings are the column names
    '''
    for feature in column_name:
        dummified_feature = pd.get_dummies(dataframe.loc[:,feature], prefix=feature, prefix_sep='__',drop_first=True)
        dummified_df = pd.concat([dataframe.drop(feature,axis=1),dummified_feature],axis=1,sort='False')
    return dummified_df

In [6]:
def delete_outliers(dataframe,column_name,threshold=5):
    '''
    - Accepts a full dataframe
    - Specify column_name to indicate which columns to check for outliers
    - Column name accepts list of strings, where the strings are the column names
    - threshold is the number of deviations that will be used for removing outliers
    '''
    for feature in column_name:
        standard_dev=np.std(dataframe.loc[:,feature])
        dataframe=dataframe.drop(dataframe[dataframe.loc[:,feature]>standard_dev*threshold].index,axis=0)
    return dataframe


In [7]:
# Check to see if nominal_var is contained in df_train
nominal_var_processed=['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass',\
             'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea']
i=0
for feature in nominal_var_processed:
    if not df_train.loc[:,feature].empty:
        i+=1
np.array(nominal_var_processed).size==i

True

In [8]:
# dummify the nominal categorical columns
df_train_copy=df_train.copy()
for feature in nominal_var_processed:
    df_train_copy=dummify_column(df_train_copy,[feature])
dummified_df=df_train_copy

# Creating an undummified dataframe for reference
undummified_df=df_train.copy()

print(dummified_df.shape)
print(dummified_df.isnull().sum().sum())
undummified_df.head(10)

(1460, 224)
0


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,No Alley,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4,3,PConc,4,3,1,GLQ,856,GasA,5,Y,SBrkr,856,854,1710,1,0,2,1,3,4,8,Typ,0,0,Attchd,2003.0,RFn,2,548,3,3,Y,0,1,0,0,0,0,No Fence,No Shed,2,2008,WD,Normal,12.247699
2,20,RL,80.0,9600,Pave,No Alley,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,CBlock,4,3,4,ALQ,1262,GasA,5,Y,SBrkr,1262,0,1262,0,1,2,0,3,3,6,Typ,1,3,Attchd,1976.0,RFn,2,460,3,3,Y,1,0,0,0,0,0,No Fence,No Shed,5,2007,WD,Normal,12.109016
3,60,RL,68.0,11250,Pave,No Alley,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,4,3,PConc,4,3,2,GLQ,920,GasA,5,Y,SBrkr,920,866,1786,1,0,2,1,3,4,6,Typ,1,3,Attchd,2001.0,RFn,2,608,3,3,Y,0,1,0,0,0,0,No Fence,No Shed,9,2008,WD,Normal,12.317171
4,70,RL,60.0,9550,Pave,No Alley,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,3,3,BrkTil,3,4,1,ALQ,756,GasA,4,Y,SBrkr,961,756,1717,1,0,1,0,3,4,7,Typ,1,4,Detchd,1998.0,Unf,3,642,3,3,Y,0,1,1,0,0,0,No Fence,No Shed,2,2006,WD,Abnorml,11.849405
5,60,RL,84.0,14260,Pave,No Alley,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,4,3,PConc,4,3,3,GLQ,1145,GasA,5,Y,SBrkr,1145,1053,2198,1,0,2,1,4,4,9,Typ,1,3,Attchd,2000.0,RFn,3,836,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,12,2008,WD,Normal,12.429220
6,50,RL,85.0,14115,Pave,No Alley,IR1,Lvl,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,3,Wood,4,3,1,GLQ,796,GasA,5,Y,SBrkr,796,566,1362,1,0,1,1,1,3,5,Typ,0,0,Attchd,1993.0,Unf,2,480,3,3,Y,1,1,0,1,0,0,MnPrv,Shed,10,2009,WD,Normal,11.870607
7,20,RL,75.0,10084,Pave,No Alley,Reg,Lvl,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,4,3,PConc,5,3,3,GLQ,1686,GasA,5,Y,SBrkr,1694,0,1694,1,0,2,0,3,4,7,Typ,1,4,Attchd,2004.0,RFn,2,636,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,8,2007,WD,Normal,12.634606
8,60,RL,80.0,10382,Pave,No Alley,IR1,Lvl,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,3,3,CBlock,4,3,2,ALQ,1107,GasA,5,Y,SBrkr,1107,983,2090,1,0,2,1,3,3,7,Typ,2,3,Attchd,1973.0,RFn,2,484,3,3,Y,1,1,1,0,0,0,No Fence,Shed,11,2009,WD,Normal,12.206078
9,50,RM,51.0,6120,Pave,No Alley,Reg,Lvl,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,3,3,BrkTil,3,3,1,Unf,952,GasA,4,Y,FuseF,1022,752,1774,0,0,2,0,2,3,8,Min1,2,3,Detchd,1931.0,Unf,2,468,2,3,Y,1,0,1,0,0,0,No Fence,No Shed,4,2008,WD,Abnorml,11.774528


In [9]:
# For all continuous features, remove outliers where outlier >= 5 std
dummified_df_copy=dummified_df.copy()
for feature in ['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','GarageArea']:
    dummified_df_copy=delete_outliers(dummified_df_copy,[feature],threshold=5)
baseline_df=dummified_df_copy 
print(baseline_df.shape)

# For the Undummified dataframe and for all continuous features, remove outliers where outlier >= 5 std
undummified_df_copy=undummified_df.copy()
for feature in ['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','GarageArea']:
    undummified_df_copy=delete_outliers(undummified_df_copy,[feature],threshold=5)
undummified_baseline_df=undummified_df_copy 
print(undummified_baseline_df.shape)

# Creating dataframes with no outlier taken out
with_outlier_dummified_baseline_df=dummified_df.copy()
with_outlier_undummified_baseline_df=undummified_df.copy()

(1215, 224)
(1215, 71)


In [10]:
# Check if ordinal variables are still there:
ordinal_var_processed=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath',\
             'HalfBath','BedroomAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars',\
             'MoSold','YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond']

i=0
for feature in ordinal_var_processed:
    if not baseline_df.loc[:,feature].empty:
        i+=1
np.array(ordinal_var_processed).size==i

True

In [11]:
# Check if continuous variables are still there:
cont_var_processed=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','TotalBsmtSF',\
          '1stFlrSF','2ndFlrSF','GrLivArea','GarageYrBlt','GarageArea']
i=0
for feature in cont_var_processed:
    if not baseline_df.loc[:,feature].empty:
        i+=1
np.array(cont_var_processed).size==i

True

In [12]:
# Standardizing/normalizing continuous and ordinal variables only
standardize_cont_var=ordinal_var_processed+cont_var_processed

standardized_baseline_df=baseline_df.copy()
standardized_baseline_df[standardize_cont_var]=baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

normalized_baseline_df=baseline_df.copy()
normalized_baseline_df[standardize_cont_var]=baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='normalize'),axis=0)

standardized_undummified_baseline_df=undummified_baseline_df.copy()
standardized_undummified_baseline_df[standardize_cont_var]=standardized_undummified_baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

normalized_undummified_baseline_df=undummified_baseline_df.copy()
normalized_undummified_baseline_df[standardize_cont_var]=normalized_undummified_baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='normalize'),axis=0)

standardized_with_outlier_dummified_baseline_df=with_outlier_dummified_baseline_df.copy()
standardized_with_outlier_dummified_baseline_df[standardize_cont_var]=standardized_with_outlier_dummified_baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

normalized_with_outlier_dummified_baseline_df=with_outlier_dummified_baseline_df.copy()
normalized_with_outlier_dummified_baseline_df[standardize_cont_var]=normalized_with_outlier_dummified_baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='normalize'),axis=0)

standardized_with_outlier_undummified_baseline_df=with_outlier_undummified_baseline_df.copy()
standardized_with_outlier_undummified_baseline_df[standardize_cont_var]=standardized_with_outlier_undummified_baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

normalized_with_outlier_undummified_baseline_df=with_outlier_undummified_baseline_df.copy()
normalized_with_outlier_undummified_baseline_df[standardize_cont_var]=normalized_with_outlier_undummified_baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='normalize'),axis=0)

print(standardized_baseline_df.shape)
print(normalized_baseline_df.shape)
print(standardized_undummified_baseline_df.shape)
print(normalized_undummified_baseline_df.shape)
print(standardized_with_outlier_dummified_baseline_df.shape)
print(normalized_with_outlier_dummified_baseline_df.shape)
print(standardized_with_outlier_undummified_baseline_df.shape)
print(normalized_with_outlier_undummified_baseline_df.shape)

# Standardizing / normalizing everything
all_standardized_baseline_df=baseline_df.copy()
all_standardized_baseline_df=all_standardized_baseline_df.apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

all_normalized_baseline_df=baseline_df.copy()
all_normalized_baseline_df=all_normalized_baseline_df.apply(lambda x:feature_standardize(x,scaleType='normalize'),axis=0)

all_standardized_with_outlier_dummified_baseline_df=with_outlier_dummified_baseline_df.copy()
all_standardized_with_outlier_dummified_baseline_df=all_standardized_with_outlier_dummified_baseline_df.apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

all_normalized_with_outlier_dummified_baseline_df=with_outlier_dummified_baseline_df.copy()
all_normalized_with_outlier_dummified_baseline_df=all_normalized_with_outlier_dummified_baseline_df.apply(lambda x:feature_standardize(x,scaleType='normalize'),axis=0)

print(all_standardized_baseline_df.shape)
print(all_normalized_baseline_df.shape)
print(all_standardized_with_outlier_dummified_baseline_df.shape)
print(all_normalized_with_outlier_dummified_baseline_df.shape)

(1215, 224)
(1215, 224)
(1215, 71)
(1215, 71)
(1460, 224)
(1460, 224)
(1460, 71)
(1460, 71)
(1215, 224)
(1215, 224)
(1460, 224)
(1460, 224)


In [13]:
baseline_df.to_csv('baseline_df.csv')
undummified_baseline_df.to_csv('undummified_baseline_df.csv')

standardized_baseline_df.to_csv('standardized_baseline_df.csv')
normalized_baseline_df.to_csv('normalized_baseline_df.csv')

standardized_undummified_baseline_df.to_csv('standardized_undummified_baseline_df.csv')
normalized_undummified_baseline_df.to_csv('normalized_undummified_baseline_df.csv')

standardized_with_outlier_dummified_baseline_df.to_csv('standardized_with_outlier_dummified_baseline_df.csv')
normalized_with_outlier_dummified_baseline_df.to_csv('normalized_with_outlier_dummified_baseline_df.csv')

standardized_with_outlier_undummified_baseline_df.to_csv('standardized_with_outlier_undummified_baseline_df.csv')
normalized_with_outlier_undummified_baseline_df.to_csv('normalized_with_outlier_undummified_baseline_df.csv')

all_standardized_baseline_df.to_csv('all_standardized_baseline_df.csv')
all_normalized_baseline_df.to_csv('all_normalized_baseline_df.csv')

with_outlier_dummified_baseline_df.to_csv('with_outlier_dummified_baseline_df.csv')    
with_outlier_undummified_baseline_df.to_csv('with_outlier_undummified_baseline_df.csv')

all_standardized_with_outlier_dummified_baseline_df.to_csv('all_standardized_with_outlier_dummified_baseline_df.csv')
all_normalized_with_outlier_dummified_baseline_df.to_csv('all_normalized_with_outlier_dummified_baseline_df.csv')